# Introduction

This tutorial demonstrates different routes to obtain phase and orientation maps from scanning electron diffraction data.

The code functionality is illustrated using synthetic data, which is first generated using pyxem. This synthetic data represents a simple bi-crystal comprising a cubic and hexagonal material adjacent to one another with two orientations, rotated by 10 degrees of each crystal. The intention is for this to provide an easy to understand illustration of the code functionality rather than to model any physical system.

This functionaility has been checked to run in pyxem-0.9.0 (July 2019). Bugs are always possible, do not trust the code blindly, and if you experience any issues please report them here: https://github.com/pyxem/pyxem-demos/issues

# Contents

1. <a href='#gen'> Setting up & Creating Synthetic Data</a>
2. <a href='#pat'> Pattern Recognition Based Mapping</a>
3. <a href='#vec'> Vector Based Mapping</a>
4. <a href='#rot'> Rotation Conventions</a>

# <a id='gen'></a> 1. Setting up & Creating Synthetic Data

Import pyxem, required libraries and pyxem modules

In [ ]:
%matplotlib tk
import numpy as np
import hyperspy.api as hs
import pyxem as pxm
import diffpy.structure
from matplotlib import pyplot as plt

from diffsims.generators.structure_library_generator import StructureLibraryGenerator
from diffsims.libraries.structure_library import StructureLibrary
from diffsims.generators.diffraction_generator import DiffractionGenerator
from diffsims.generators.library_generator import DiffractionLibraryGenerator, VectorLibraryGenerator

from pyxem.generators.indexation_generator import IndexationGenerator
from pyxem.generators.indexation_generator import VectorIndexationGenerator

from pyxem.utils.sim_utils import sim_as_signal
from pyxem.utils.indexation_utils import peaks_from_best_template
from pyxem.utils.plot import generate_marker_inputs_from_peaks

Define two illustrative crystal structures

In [ ]:
latt = diffpy.structure.lattice.Lattice(5, 5, 5, 90, 90, 90)
atom = diffpy.structure.atom.Atom(atype='Si', xyz=[0, 0, 0], lattice=latt)
si = diffpy.structure.Structure(atoms=[atom], lattice=latt)

In [ ]:
latt = diffpy.structure.lattice.Lattice(3, 3, 5, 90, 90, 120)
atom = diffpy.structure.atom.Atom(atype='Ga', xyz=[0, 0, 0], lattice=latt)
ga = diffpy.structure.Structure(atoms=[atom], lattice=latt)

Define simulation paramaters and initialize DiffractionGenerator

In [ ]:
pattern_size = 256  # pixels
half_pattern_size = pattern_size // 2
reciprocal_radius = 1.2
calibration = reciprocal_radius / half_pattern_size
beam_energy = 300.0

ediff = DiffractionGenerator(beam_energy, 0.025)  # keV and relrod length (1/Å)

Create 4 seperate patterns, 2 for each crystal, one at 0 degress and one at 10 degrees.

In [ ]:
phase_names = ['Si', 'Ga'] 
sample_lib = StructureLibrary(phase_names, [si, ga], [
                                  [(10,0,0), (0,0,0)],  # For Si
                                  [(0,0,0), (10,0,0)]   # For Ga
                              ])

diff_gen = DiffractionLibraryGenerator(ediff)
library = diff_gen.get_diffraction_library(sample_lib,
                                           calibration=calibration,
                                           reciprocal_radius=reciprocal_radius,
                                           half_shape=(half_pattern_size, half_pattern_size),
                                           with_direct_beam=False)

data_silicon = []
data_gallium = []

for theta in [0, 10]:
    pattern = sim_as_signal(library.get_library_entry(phase='Si', angle=(theta, 0, 0))['Sim'],
                            pattern_size, 0.03, reciprocal_radius)
    data_silicon.append(pattern)
    pattern = sim_as_signal(library.get_library_entry(phase='Ga', angle=(theta, 0, 0))['Sim'],
                            pattern_size, 0.03, reciprocal_radius)
    data_gallium.append(pattern)
        
data = [x.data for x in data_silicon] + [x.data for x in data_gallium]

test_data = pxm.ElectronDiffraction2D(np.asarray(data).reshape(2, 2, pattern_size, pattern_size))
test_data.set_diffraction_calibration(calibration) 

Plot the synethetic data that is analyzed in subsequent sections.

In [ ]:
test_data.plot(cmap='viridis')

#  <a id='pat'></a> 2. Pattern Recognition Based Mapping

The pattern recognition approach involves simulating a library of theoretical diffraction patterns for expected phases and orientations and then determining the best fitting simulation for each experimental pattern. Conventions for specifying crystal orientations are specified here: http://pyxem.github.io/pyxem/conventions.html

Define a StructureLibrary containing specifying two crystal structures and orientations of each to include. The in-plane orientations can be a list of known options (set `inplane_rotations=[0]` in the call to `correlate` below) or the starting points for the list of rotations given to `correlate` below.

In [ ]:
structure_library_generator = StructureLibraryGenerator(
    [('Si', si, 'cubic'),
     ('Ga', ga, 'hexagonal')])
structure_library = structure_library_generator.get_orientations_from_stereographic_triangle(
    [(0,), (0,)],  # In-plane rotations
    5)  # Angular resolution of the library

Generate a library of simulated diffraction data

In [ ]:
diff_gen = DiffractionLibraryGenerator(ediff)
template_library = diff_gen.get_diffraction_library(structure_library,
                                                    calibration=calibration,
                                                    reciprocal_radius=reciprocal_radius-0.1,
                                                    half_shape=(half_pattern_size, half_pattern_size),
                                                    with_direct_beam=False)

Correlate with the patterns contained in the library with the test data. At this stage the top 3 (`n_largest`) matching results are retained. Test all in-plane rotations at 5 degree increments from 0 to 360.

In [ ]:
indexer = IndexationGenerator(test_data, template_library)
match_results = indexer.correlate(n_largest=3, inplane_rotations=np.arange(0, 360, 5))

We now have a range of ways of working with this output, but here we simply plot it to show that the method has worked as anticipated.

In [ ]:
match_results.plot_best_matching_results_on_signal(test_data, template_library, permanent_markers=False, cmap='viridis')

Obtain an orientation map from the matching results, by selecting the top matching result, and plot it.

In [ ]:
cryst_map = match_results.get_crystallographic_map()
ori_map = cryst_map.get_orientation_map()
ori_map.plot(cmap='inferno')

#  <a id='vec'></a> 3. Vector Based Mapping

The vector matching approach involves finding the diffraction vectors measured within each pattern, using peak finding methods, mapping these 2D detector coordinates to 3D reciprocal space coordinates and then matching these peaks against theoretical reciprocal lattice vectors for anticipated crystal phases present.

Find peaks using the cross correlation `'xc'` method and map detector coordinates to reciprocal space coordinates.

In [ ]:
peak_example = test_data.inav[0, 0].isig[122:135, 101:114].data
peaks = test_data.find_peaks('xc', disc_image=peak_example, peak_threshold=0.8)
peaks.calculate_cartesian_coordinates(beam_energy, 0.2)  # Camera length in meters

Define a structure library for the expected phases from section 1.

In [ ]:
structure_library = StructureLibrary(['Si', 'Ga'], [si, ga], [[], []])

Calulate a VectorLibrary containing expected reciprocal lattice vectors (and their pairs with inter-vector angles) for each expected phase.

In [ ]:
library_generator = VectorLibraryGenerator(structure_library)
vector_library = library_generator.get_vector_library(reciprocal_radius)

Perform indexation by looking for peak pairs with vector lengths within 1.5 pixels and angle within 1° of the experimental peaks.

In [ ]:
indexation_generator = VectorIndexationGenerator(peaks, vector_library)
indexation = indexation_generator.index_vectors(mag_tol=1.5*calibration,
                                                angle_tol=1,
                                                index_error_tol=0.2,
                                                n_peaks_to_index=5,
                                                n_best=2)

Calculate a crystallographic map containing the indexation results for further manipulation

In [ ]:
cryst_map = indexation.get_crystallographic_map()

Plot indexed peaks on top of the data using the template library from above to generate peak positions.

In [ ]:
indexation.plot_best_matching_results_on_signal(test_data, template_library, permanent_markers=False)

Obtain an orientation map from the matching results, by selecting the top matching result, and plot it.

In [ ]:
cryst_map = indexation.get_crystallographic_map()
ori_map = cryst_map.get_orientation_map()
ori_map.plot(cmap='inferno')

Plot the match rate as a metric for mapping success.

In [ ]:
cryst_map.get_metric_map('match_rate').plot()

# <a id='rot'></a>  4. Rotation Conventions

To illustrate the rotation convention used, diffraction patterns for two orientations "f1" and "f2" are plotted below.

In [ ]:
f1 = (90, -3, -90)
f2 = (90, +3, -90)
sample_lib = StructureLibrary(['Si'], [si], [[f1, f2]])
diff_gen = DiffractionLibraryGenerator(ediff)
library = diff_gen.get_diffraction_library(sample_lib,
                                           calibration=1 / 64,
                                           reciprocal_radius=0.8,
                                           half_shape=(64,64),
                                           with_direct_beam=False)

pattern = sim_as_signal(library.get_library_entry(phase='Si', angle=f1)['Sim'],
                        pattern_size, 0.03, 1)
plt.figure('f1')
plt.imshow(pattern, cmap='viridis', vmax=0.1)

pattern = sim_as_signal(library.get_library_entry(phase='Si', angle=f2)['Sim'],
                        pattern_size, 0.03, 1)
plt.figure('f2')
plt.imshow(pattern, cmap='viridis', vmax=0.1)